In [ ]:
#| default_exp gst_valve

In [ ]:
# ---
# skip_showdoc: true
# ---

# Gstreamer Python Bindings

> Gstreamer video capture


In [ ]:
#| export
from fastcore.utils import *
from fastcore.utils import *
import cv2
import gi
import numpy as np
from imutils import resize
# from ping_ip import ping_ip

from gi.repository import Gst
import subprocess
import platform

import paho.mqtt.client as mqtt

import time
# from dataloader import LoadImages, resize
from pathlib import Path
# import .nbs.Gstreamer.gst_parameters as params

In [ ]:
#| export
# gi.require_version('Gst', '1.0')

In [ ]:
#| hide
from nbdev.showdoc import *
from nbdev.showdoc import *
from fastcore.test import *


### Default parameters
see the code below

In [ ]:
#| exports
#|code-fold: true
from dataclasses import dataclass

@dataclass
class DefaultParams():
    camera_dev = "CAM-0"
    cameras = {
        "CAM-0": {
            "gst": [
                'videotestsrc pattern=smpte is-live=true ! tee name=t ',
                't. ! queue leaky=2 ! videoconvert ! videorate drop-only=true ! video/x-raw,framerate=10/1,format=(string)BGR ! videoconvert ! appsink emit-signals=true  sync=false async=false  max-buffers=2 drop=true ',
                't. ! queue leaky=2 ! valve name=myvalve drop=true ! video/x-raw,format=I420,width=640,height=480 ! videoconvert ! x264enc ! rtph264pay ! udpsink host=127.0.0.1 port=5000',
                ],
            "udp": True,
            "host": "127.0.0.1",
            "port": 5000,
        },
        "CAM-1": {
            "gst": [
                'videotestsrc pattern=ball is-live=true ! tee name=t ',
                't. ! queue leaky=2 ! videoconvert ! videorate drop-only=true ! video/x-raw,framerate=10/1,format=(string)BGR ! videoconvert ! appsink emit-signals=true  sync=false async=false  max-buffers=2 drop=true ',
                't. ! queue leaky=2 ! valve name=myvalve drop=true ! video/x-raw,format=I420,width=640,height=480 ! videoconvert ! x264enc ! rtph264pay ! udpsink host=127.0.0.1 port=5000',
                ],
            "udp": True,
            "host": "127.0.0.1",
            "port": 5000,
        },
        "CAM-2": {
            "gst": [
                'videotestsrc pattern=snow is-live=true ! tee name=t ',
                't. ! queue leaky=2 ! videoconvert ! videorate drop-only=true ! video/x-raw,framerate=10/1,format=(string)BGR ! videoconvert ! appsink emit-signals=true  sync=false async=false  max-buffers=2 drop=true ',
                't. ! queue leaky=2 ! valve name=myvalve drop=true ! video/x-raw,format=I420,width=640,height=480 ! videoconvert ! x264enc ! rtph264pay ! udpsink host=127.0.0.1 port=5000',
                ],
            "udp": True,
            "host": "127.0.0.1",
            "port": 5000,
        },
        "CAM-3": {
            "gst": [
                'videotestsrc pattern=pinwheel is-live=true ! tee name=t ',
                't. ! queue leaky=2 ! videoconvert ! videorate drop-only=true ! video/x-raw,framerate=10/1,format=(string)BGR ! videoconvert ! appsink emit-signals=true  sync=false async=false  max-buffers=2 drop=true ',
                't. ! queue leaky=2 ! valve name=myvalve drop=true ! video/x-raw,format=I420,width=640,height=480 ! videoconvert ! x264enc ! rtph264pay ! udpsink host=127.0.0.1 port=5000',
                ],
            "udp": True,
            "host": "127.0.0.1",
            "port": 5000,
            },
    
       }

    # socket address and port
    mqqt_address='127.0.0.1'
    src_port=1234

```DefaultParams.cameras["CAM-0"]["gst"]``` is a list of gstreamer commands.

In [ ]:
print(DefaultParams.cameras["CAM-0"])

{'gst': ['videotestsrc pattern=smpte is-live=true ! tee name=t ', 't. ! queue leaky=2 ! videoconvert ! videorate drop-only=true ! video/x-raw,framerate=10/1,format=(string)BGR ! videoconvert ! appsink emit-signals=true  sync=false async=false  max-buffers=2 drop=true ', 't. ! queue leaky=2 ! valve name=myvalve drop=true ! video/x-raw,format=I420,width=640,height=480 ! videoconvert ! x264enc ! rtph264pay ! udpsink host=127.0.0.1 port=5000'], 'udp': True, 'host': '127.0.0.1', 'port': 5000}


The default parameters list 4 gst videotestsrc each with a different pattern. 
The patterns are: 
- smpte,  ball, snow, pinwheel.

The video is split with a tee into two streams 
1.  is streamed via an on / off valve to  udp port 5000.
2. is streamed to an appsink for processing.


### Gstreamer pipeline

In [ ]:
#| exports
#|code-fold: true
class GstVideoStream():
    "Video streamer class using gstreamer"
    def __init__(self, gstcommand:List  # gst command list
                 , address:str='127.0.0.1'  # udp address
                 , port:int=5000): # udp port
        Gst.init(None)
        self.gstcommand = gstcommand
        self.address = address
        self.port = port

        self.latest_frame = self._new_frame = None
        self.video_pipe = None
        self.video_sink = None
        self.pause = False
        self.run()

    def start_gst(self, config=None):
        """ Start gstreamer pipeline and sink
        """
        command = ' '.join(self.gstcommand)
        print(command)
        self.video_pipe = Gst.parse_launch(command)
        self.video_pipe.set_state(Gst.State.PLAYING)
        self.video_sink = self.video_pipe.get_by_name('appsink0')

    @staticmethod
    def gst_to_opencv(sample):
        "Transform byte array into np array"

        buf = sample.get_buffer()
        caps_structure = sample.get_caps().get_structure(0)
        array = np.ndarray(
            (
                caps_structure.get_value('height'),
                caps_structure.get_value('width'),
                3
            ),
            buffer=buf.extract_dup(0, buf.get_size()), dtype=np.uint8)
        return array

    def frame(self):
        """ Get Frame
        Returns:
            np.ndarray: latest retrieved image frame
        """
        if self.frame_available:
            self.latest_frame = self._new_frame
            # reset to indicate latest frame has been 'consumed'
            self._new_frame = None
        return self.latest_frame

    # def frame_available(self):
    #     """Check if a new frame is available
    #     Returns:
    #         bool: true if a new frame is available
    #     """
    #     return self._new_frame is not None

    def run(self):
        """ Get frame to update _new_frame
        """

        self.start_gst()
        try:
            self.video_sink.connect('new-sample', self.callback)
        except:
            pass

    def callback(self, sink):
        sample = sink.emit('pull-sample')
        # if not self.pause:
        self._new_frame = self.gst_to_opencv(sample)

        return Gst.FlowReturn.OK


In [ ]:
#| export
@patch
def frame_available(self:GstVideoStream):
    "Check and return frame is available"
    return self._new_frame is not None

### Valve gives the ability to pause the video stream

In [ ]:
#| export
def toggle_valve_state(pipeline:Gst.Pipeline  # set up by Gst.parse_launch(command)
                       , valvename: str):  # name of valve element
    " Toggle the state of a valve element"

    valve = pipeline.get_by_name(valvename)
    current_drop_state = valve.get_property("drop")
    print("current_drop_state", current_drop_state)
    valve.set_property("drop", not current_drop_state)
    current_drop_state = valve.get_property("drop")
    print("new_drop_state", current_drop_state)

    # valve.set_property("drop", state)
def set_valve_state(pipeline:Gst.Pipeline  # set up by Gst.parse_launch(command)
                    , valvename: str  # name of valve element
                    , drop_state: bool  # True = drop frames
                    ):
    "Set the state of a valve element"
    
    valve = pipeline.get_by_name(valvename)
    current_drop_state = valve.get_property("drop")
    print("current_drop_state", current_drop_state)
    valve.set_property("drop", drop_state)
    current_drop_state = valve.get_property("drop")
    print("new_drop_state", current_drop_state)



In [ ]:
#| export
def ping_ip(ip_address:str # IP address to ping
            )->bool :  # returns True if IP address is in use
    "Ping an IP address to see if it is in use"
    if platform.system().lower() == "windows":
        status = subprocess.call(
            ['ping', '-q', '-n', '1', '-W', '1', ip_address],
            stdout=subprocess.DEVNULL)
    else:
        status = subprocess.call(
            ['ping', '-q', '-c', '1', '-W', '1', ip_address],
            stdout=subprocess.DEVNULL)

    if status == 0:
        return True
    else:
        return False

Test ping IP

In [ ]:
test_eq(ping_ip("127.0.0.1"), True)

In [ ]:
test_eq(ping_ip("1.2.3.4"), False)

In [ ]:
#| export
class Mqtt:
    "Class to control the valve via MQTT"
    def __init__(self, camera:str  # name of camera
                 , video:GstVideoStream  # video object
                 ):
        self.camera = camera
        self.video = video
        self.client = mqtt.Client(self.camera)

        addr = "10.42.0.1"
        if ping_ip(addr):
            print("Connecting to ", addr)
            self.client.connect(addr)
        else:
            print("Connecting to ", "127.0.0.1")
            self.client.connect("127.0.0.1")

        self.client.loop_start()
        self.client.subscribe("STREAM-CAMERA")
        self.client.on_message = self.on_mqtt_message

    def on_mqtt_message(self, client:mqtt.Client # mqtt client
                        , userdata # user data
                        , message:mqtt.MQTTMessage # message
                        ):
        mess = str(message.payload.decode("utf-8"))
        print("Received message: ", mess)
        if mess == self.camera:
            set_valve_state(self.video.video_pipe, "myvalve", False)
        else:
            set_valve_state(self.video.video_pipe, "myvalve", True)
    
    def close(self):
        self.client.loop_stop()
        self.client.disconnect()
        print("Closed mqtt client")


In [ ]:

show_doc(Mqtt)

---

### Mqtt

>      Mqtt (camera:str, video:__main__.VideoStreamer)

Class to control the valve via MQTT

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| camera | str | name of camera |
| video | VideoStreamer | video object |

### Main function for local testing

In [ ]:
#| exports
#|code-fold: true
def main(camera="CAM-0"):
    params = DefaultParams()
    cv2.namedWindow(camera, cv2.WINDOW_NORMAL)
    gstcommand = params.cameras[camera]["gst"]
    video = GstVideoStream(gstcommand)
    mqtt = Mqtt(camera, video)

    print('Initialising stream...')
    waited = 0
    while not video.frame_available():
        waited += 1
        print('\r  Frame not available (x{})'.format(waited), end='')
        cv2.waitKey(30)

    print('\nSuccess!\nStarting streaming - press "q" to quit.')

    wait_time = 1
    count = 0
    while True:

        if video.frame_available() and count % 10 == 0:
            frame = video.frame().copy()
            # # cv2.putText(frame, f'{frame_num:2d} {data_received}', (10, 30), cv2.FONT_HERSHEY_PLAIN, 1, (0, 0, 255), 2)
            frame = resize(frame, width= 600)
            cv2.imshow(camera, frame)
            pass


        if count % 1000 == 0:
            print( count)
        count += 1


        k = cv2.waitKey(wait_time)

        if k == ord('q') or k == ord('Q') or k == 27:
            break

        if k == ord('v'):
            # Assuming you have a valve element named 'myvalve' in your pipeline
            valve = video.video_pipe.get_by_name("myvalve")
            current_drop_state = valve.get_property("drop")
            print("current_drop_state", current_drop_state)
            valve.set_property("drop", not current_drop_state)
            current_drop_state = valve.get_property("drop")
            print("new_drop_state", current_drop_state)

            time.sleep(2)

            # video.video_pipe.set_state(Gst.State.PLAYING)

        if k == ord(' '):
            if wait_time != 0:
                wait_time = 0
            else:
                wait_time = 1

        if k == ord('s'):
            save = 0
            save_path = Path(params.save_path) 
            save_path.mkdir(exist_ok=True)
            pass

    mqtt.close()


### Receive on UDP with this Test  :
from first terminal run 
```sh
   gst-launch-1.0 udpsrc port=5000 ! application/x-rtp,encoding-name=H264,payload=96 ! \
           rtph264depay ! h264parse ! queue ! avdec_h264 ! xvimagesink sync=false async=false -e
  ```
from second terminal run 
```sh
  mosquitto_pub -m "CAM-0" -t "STREAM-CAMERA"
  mosquitto_pub -m "CAM-1" -t "STREAM-CAMERA"
```

### Test with two cameras from terminal

In [ ]:
from multiprocessing import Process #|hide_line

In [ ]:
#| exports
#|eval: false     don't run this cell in testing
#|code-fold: true
# from multiprocessing import Process   # you will need to import Process from multiprocessing

try: __name__
except NameError: __name__ = None   # stops nbdev_preview from forever looping / crashing
if __name__ == '__main__':
    
    cams = []
    params = DefaultParams()
    for cam in list(params.cameras.keys())[:2]:
        print(cam)
        p = Process(target=main, args=(cam,))
        p.start()
        cams.append(p)

    for p in cams:
        p.join()

CAM-0
CAM-1
videotestsrc pattern=ball is-live=true ! tee name=t  t. ! queue leaky=2 ! videoconvert ! videorate drop-only=true ! video/x-raw,framerate=10/1,format=(string)BGR ! videoconvert ! appsink emit-signals=true  sync=false async=false  max-buffers=2 drop=true  t. ! queue leaky=2 ! valve name=myvalve drop=true ! video/x-raw,format=I420,width=640,height=480 ! videoconvert ! x264enc ! rtph264pay ! udpsink host=127.0.0.1 port=5000videotestsrc pattern=smpte is-live=true ! tee name=t  t. ! queue leaky=2 ! videoconvert ! videorate drop-only=true ! video/x-raw,framerate=10/1,format=(string)BGR ! videoconvert ! appsink emit-signals=true  sync=false async=false  max-buffers=2 drop=true  t. ! queue leaky=2 ! valve name=myvalve drop=true ! video/x-raw,format=I420,width=640,height=480 ! videoconvert ! x264enc ! rtph264pay ! udpsink host=127.0.0.1 port=5000
Connecting to Connecting to   127.0.0.1
127.0.0.1
Initialising stream...
Initialising stream...
Success!
Starting streaming - press "q" to

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()